In [1]:
#import dependencies
import pandas as pd
import numpy as np
import os
import gmaps
import json
import requests
from pprint import pprint
import us

#import API key
# from config_template import gkey, weather_api_key
from config import gkey, weather_api_key
gmaps.configure(api_key=gkey)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

## Gather Data for the Cities Sunshine

- mannually imported information from text file online to csv
- mannually did some cleaning with the data in excel
- we do not know web scraping yet so was unable to utilize that for infomration

In [2]:
#read the sunshine csv file into a dataframe
sunshine_df = pd.read_csv(os.path.join("..","potential_data_sets","NOAA_Sunshine_AveragePercentPossible.csv"))
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90%,42
1,REDDING,CA,88%,10
2,LAS VEGAS,NV,85%,47
3,PHOENIX,AZ,85%,101
4,TUCSON,AZ,85%,53
...,...,...,...,...
169,POHNPEI,FM,41%,44
170,ELKINS,WV,40%,11
171,MT. WASHINGTON,NH,33%,64
172,QUILLAYUTE,WA,33%,30


In [3]:
#clean up the column names
sunshine_df = sunshine_df.rename(columns={
    "CITY":"City",
    "STATE":"State",
    "ANNUAL % AVG POSSIBLE SUNSHINE":"Annual % Average Possible Sunshine",
    "NO. OF YEARS OF DATA":"# of Data Years"
})

In [4]:
#check the datatypes
sunshine_df.dtypes

City                                  object
State                                 object
Annual % Average Possible Sunshine    object
# of Data Years                        int64
dtype: object

In [5]:
#strip the % column of the % symbol and convert to float
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].map(lambda x: x.rstrip('%'))
sunshine_df["Annual % Average Possible Sunshine"] = sunshine_df["Annual % Average Possible Sunshine"].astype(float)
sunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,YUMA,AZ,90.0,42
1,REDDING,CA,88.0,10
2,LAS VEGAS,NV,85.0,47
3,PHOENIX,AZ,85.0,101
4,TUCSON,AZ,85.0,53
...,...,...,...,...
169,POHNPEI,FM,41.0,44
170,ELKINS,WV,40.0,11
171,MT. WASHINGTON,NH,33.0,64
172,QUILLAYUTE,WA,33.0,30


In [31]:
sunshine_df.dtypes

City                                   object
State                                  object
Annual % Average Possible Sunshine    float64
# of Data Years                         int64
dtype: object

- State dictionary credit to: https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
- Added in some territories mannually to this dictionary

In [6]:
#dictionary of state abbreviations:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'FM': 'Federal States of Micronesia',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MH': 'Marshall Islands',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'PW': 'Palau',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [7]:
state=[]
for row in range(len(sunshine_df)):
    try:
        state.append(states_dict[sunshine_df['State'][row]])
    except:
        state.append(sunshine_df['State'][row])

cleanSunshine_df = sunshine_df.copy()
cleanSunshine_df["State"]=state
cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,YUMA,Arizona,90.0,42
1,REDDING,California,88.0,10
2,LAS VEGAS,Nevada,85.0,47
3,PHOENIX,Arizona,85.0,101
4,TUCSON,Arizona,85.0,53
...,...,...,...,...
169,POHNPEI,Federal States of Micronesia,41.0,44
170,ELKINS,West Virginia,40.0,11
171,MT. WASHINGTON,New Hampshire,33.0,64
172,QUILLAYUTE,Washington,33.0,30


In [8]:
#clean the casing of cities
cities = cleanSunshine_df['City']
cities = cities.str.title()
cleanSunshine_df['City']=cities
cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years
0,Yuma,Arizona,90.0,42
1,Redding,California,88.0,10
2,Las Vegas,Nevada,85.0,47
3,Phoenix,Arizona,85.0,101
4,Tucson,Arizona,85.0,53
...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44
170,Elkins,West Virginia,40.0,11
171,Mt. Washington,New Hampshire,33.0,64
172,Quillayute,Washington,33.0,30


In [9]:
#google maps geocoding url
url_geocoding = "https://maps.googleapis.com/maps/api/geocode/json?"

#find the latitude and longitude of the cities
lat = []
lng = []

#define countrycode
currentCountry = "US"

#loop through the cities_df to retrieve the weather information
for index, row in cleanSunshine_df.iterrows():
    currentCity = row["City"]
    currentState = row["State"]
    target_url = f"{url_geocoding}address={currentCity},{currentState}&region={currentCountry}&key={gkey}"
    response = requests.get(target_url).json()
    
    try:
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lng.append(response['results'][0]['geometry']['location']['lng'])
        print(f"Processeing Record | {currentCity}")
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print(f"City not found. Skipping {currentCity}...")

#update latitude and longitude to actually match the city
cleanSunshine_df["Latitude"] = lat
cleanSunshine_df["Longitude"] = lng

Processeing Record | Yuma
Processeing Record | Redding
Processeing Record | Las Vegas
Processeing Record | Phoenix
Processeing Record | Tucson
Processeing Record | El Paso
Processeing Record | Fresno
Processeing Record | Reno
Processeing Record | Flagstaff
Processeing Record | Sacramento
Processeing Record | Albuquerque
Processeing Record | Key West
Processeing Record | Pueblo
Processeing Record | Midland-Odessa
Processeing Record | Roswell
Processeing Record | Amarillo
Processeing Record | Ely
Processeing Record | Los Angeles
Processeing Record | Lake Charles
Processeing Record | Lubbock
Processeing Record | North Little Rock
Processeing Record | Grand Junction
Processeing Record | Honolulu
Processeing Record | Johnston Island
Processeing Record | Abilene
Processeing Record | Dodge City
Processeing Record | Miami
Processeing Record | Milford
Processeing Record | Wake Island
Processeing Record | Denver
Processeing Record | Lander
Processeing Record | Oklahoma City
Processeing Record | 

## Export Full Cleaned City Sunshine Dataframe

In [10]:
#export results to csv file
cleanSunshine_df.to_csv(os.path.join(outputFilePath,"cleanSunshine.csv"),index=False)

cleanSunshine_df

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
...,...,...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44,6.854125,158.262382
170,Elkins,West Virginia,40.0,11,38.925940,-79.846735
171,Mt. Washington,New Hampshire,33.0,64,44.270585,-71.303272
172,Quillayute,Washington,33.0,30,47.943130,-124.542435


In [11]:
#clean the data set of null values
sunshineLocations_df = cleanSunshine_df.dropna().reset_index(drop=True)
sunshineLocations_df

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
...,...,...,...,...,...,...
169,Pohnpei,Federal States of Micronesia,41.0,44,6.854125,158.262382
170,Elkins,West Virginia,40.0,11,38.925940,-79.846735
171,Mt. Washington,New Hampshire,33.0,64,44.270585,-71.303272
172,Quillayute,Washington,33.0,30,47.943130,-124.542435


## Sunshine by City Heatmap

In [12]:
#collect the weights of %
sunshine = sunshineLocations_df["Annual % Average Possible Sunshine"].astype(float)

#max of % sunshine
maxSun = sunshineLocations_df["Annual % Average Possible Sunshine"].max()

#locations of sun for heat
locations = sunshineLocations_df[["Latitude","Longitude"]].astype(float)

In [13]:
#create the sunshine map
figHeatCity = gmaps.figure()

#create the heat layer
heatLayer_sunshine = gmaps.heatmap_layer(locations, weights=sunshine, max_intensity=maxSun, dissipating=True)

#add the heat layer
figHeatCity.add_layer(heatLayer_sunshine)

#display the figure
figHeatCity

Figure(layout=FigureLayout(height='420px'))

## Export the Top City Sunshine Results

In [14]:
#find locations with sun >= 70%
topSunshine_df = sunshineLocations_df.loc[(sunshineLocations_df["Annual % Average Possible Sunshine"]>=70)]
topSunshine_df.head()

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711


In [15]:
#export results to csv file
topSunshine_df.to_csv(os.path.join(outputFilePath,"topCitySunshine.csv"),index=False)

## Top City Sunshine w/ Heatmap + Markers

In [16]:
#DO NOT CHANGE CODE IN THIS CELL

#using the infobox template
info_box_template = """
<dl>
<dt>State</dt><dd>{State}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>% Sunshine</dt><dd>{Annual % Average Possible Sunshine}</dd>
</dl>
"""

#Store the dataframe row
#update with names of dataframe
topSunshine_info = [info_box_template.format(**row) for index, row in topSunshine_df.iterrows()]
topSunshine_markerLocations = topSunshine_df[['Latitude','Longitude']]

In [17]:
#add marker layer onto heatmap
markers = gmaps.marker_layer(topSunshine_markerLocations,info_box_content=topSunshine_info)
figHeatCity.add_layer(markers)

#display figure
figHeatCity

Figure(layout=FigureLayout(height='420px'))

In [18]:
#top 10 cities avg sunshine
topSunshine_df.nlargest(10,["Annual % Average Possible Sunshine"])

,City,State,Annual % Average Possible Sunshine,# of Data Years,Latitude,Longitude
0,Yuma,Arizona,90.0,42,32.692651,-114.627692
1,Redding,California,88.0,10,40.586540,-122.391675
2,Las Vegas,Nevada,85.0,47,36.169941,-115.139830
3,Phoenix,Arizona,85.0,101,33.448377,-112.074037
4,Tucson,Arizona,85.0,53,32.222607,-110.974711
5,El Paso,Texas,84.0,54,31.761878,-106.485022
6,Fresno,California,79.0,46,36.737798,-119.787125
7,Reno,Nevada,79.0,45,39.529633,-119.813803
8,Flagstaff,Arizona,78.0,15,35.198284,-111.651302
9,Sacramento,California,78.0,46,38.581572,-121.494400


## Gather Data for State Summary

In [19]:
#read the sunshine csv file into a dataframe
states_df = pd.read_csv(os.path.join("..","potential_data_sets","50_us_states_all_data.csv"))
del states_df["CAPS STATE NAME"]
del states_df["ABBREV."]
states_df = states_df.rename(columns = {
    "TITLE STATE NAME":"State",
    "2 LETTER ABBREVIATION":"State Abbreviation"
})
states_df.head()

,State,State Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [20]:
#google maps geocoding url
url_geocoding = "https://maps.googleapis.com/maps/api/geocode/json?"

#find the latitude and longitude of the cities
stateLat = []
stateLng = []

#define countrycode
currentCountry = "US"

#loop through the cities_df to retrieve the weather information
for index, row in states_df.iterrows():
    currentState = row["State"]
    target_url = f"{url_geocoding}address={currentState}&region={currentCountry}&key={gkey}"
    response = requests.get(target_url).json()
    
    try:
        stateLat.append(response['results'][0]['geometry']['location']['lat'])
        stateLng.append(response['results'][0]['geometry']['location']['lng'])
        print(f"Processeing Record | {currentState}")
    except:
        stateLat.append(np.nan)
        stateLng.append(np.nan)
        print(f"City not found. Skipping {currentState}...")

#update latitude and longitude to actually match the city
states_df["Latitude"] = stateLat
states_df["Longitude"] = stateLng

Processeing Record | Alabama
Processeing Record | Alaska
Processeing Record | Arizona
Processeing Record | Arkansas
Processeing Record | California
Processeing Record | Colorado
Processeing Record | Connecticut
Processeing Record | Delaware
Processeing Record | Florida
Processeing Record | Georgia
Processeing Record | Hawaii
Processeing Record | Idaho
Processeing Record | Illinois
Processeing Record | Indiana
Processeing Record | Iowa
Processeing Record | Kansas
Processeing Record | Kentucky
Processeing Record | Louisiana
Processeing Record | Maine
Processeing Record | Maryland
Processeing Record | Massachusetts
Processeing Record | Michigan
Processeing Record | Minnesota
Processeing Record | Mississippi
Processeing Record | Missouri
Processeing Record | Montana
Processeing Record | Nebraska
Processeing Record | Nevada
Processeing Record | New Hampshire
Processeing Record | New Jersey
Processeing Record | New Mexico
Processeing Record | New York
Processeing Record | North Carolina
Proc

In [21]:
states_df.head()

,State,State Abbreviation,Latitude,Longitude
0,Alabama,AL,32.318231,-86.902298
1,Alaska,AK,64.200841,-149.493673
2,Arizona,AZ,34.048928,-111.093731
3,Arkansas,AR,35.201050,-91.831833
4,California,CA,36.778261,-119.417932


In [22]:
#gather the average of each state sunshine from sunshineLocation_df 'Annual % Average Possible Sunshine'
stateSunAvg_df = sunshineLocations_df.groupby('State')['Annual % Average Possible Sunshine'].mean()
stateSunAvg_df = stateSunAvg_df.reset_index()
stateSunAvg_df.head()

,State,Annual % Average Possible Sunshine
0,Alabama,57.666667
1,Alaska,37.666667
2,American Samoa,42.000000
3,Arizona,84.500000
4,Arkansas,65.000000


In [23]:
#merge the avg sun per state with the states_df
statesInfo_df = states_df.merge(stateSunAvg_df,on="State")
statesInfo_df.head()

,State,State Abbreviation,Latitude,Longitude,Annual % Average Possible Sunshine
0,Alabama,AL,32.318231,-86.902298,57.666667
1,Alaska,AK,64.200841,-149.493673,37.666667
2,Arizona,AZ,34.048928,-111.093731,84.500000
3,Arkansas,AR,35.201050,-91.831833,65.000000
4,California,CA,36.778261,-119.417932,71.857143


## Export the State Summary

In [24]:
#export results to csv file
statesInfo_df.to_csv(os.path.join(outputFilePath,"stateSunshine.csv"),index=False)

## Sunshine by State Average

In [25]:
#collect the weightsof %
sunshine = statesInfo_df["Annual % Average Possible Sunshine"].astype(float)

#max of % sunshine
maxSun = statesInfo_df["Annual % Average Possible Sunshine"].max()

#locations of sun for heat
locations = statesInfo_df[["Latitude","Longitude"]].astype(float)

In [26]:
#create the sunshine map
figStates = gmaps.figure()

#create the heat layer
heatLayer_sunshine = gmaps.heatmap_layer(locations, weights=sunshine, max_intensity=maxSun, dissipating=True)

#add the heat layer
figStates.add_layer(heatLayer_sunshine)

#display the figure
figStates

Figure(layout=FigureLayout(height='420px'))

## Avg State Sunshine w/ Heatmap + Markers

In [27]:
#DO NOT CHANGE CODE IN THIS CELL

#using the infobox template
info_box_template = """
<dl>
<dt>State</dt><dd>{State}</dd>
<dt>% Sunshine</dt><dd>{Annual % Average Possible Sunshine}</dd>
</dl>
"""

#Store the dataframe row
#update with names of dataframe
stateSunshine_info = [info_box_template.format(**row) for index, row in statesInfo_df.iterrows()]
stateSunshine_markerLocations = statesInfo_df[['Latitude','Longitude']]

In [28]:
#create 1 info box per state w/ all cities and sunshine information
#add marker layer onto heatmap
markers = gmaps.marker_layer(stateSunshine_markerLocations,info_box_content=stateSunshine_info)
figStates.add_layer(markers)

#display figure
figStates

Figure(layout=FigureLayout(height='420px'))

In [29]:
#top 10 states avg sunshine
statesInfo_df.nlargest(10,["Annual % Average Possible Sunshine"])

,State,State Abbreviation,Latitude,Longitude,Annual % Average Possible Sunshine
2,Arizona,AZ,34.048928,-111.093731,84.500000
26,Nevada,NV,38.802610,-116.419389,76.250000
29,New Mexico,NM,34.519940,-105.870090,75.000000
5,Colorado,CO,39.550051,-105.782067,72.000000
4,California,CA,36.778261,-119.417932,71.857143
42,Utah,UT,39.320980,-111.093731,68.000000
7,Florida,FL,27.664827,-81.515754,66.833333
14,Kansas,KS,39.011902,-98.484246,65.750000
48,Wyoming,WY,43.075968,-107.290284,65.666667
41,Texas,TX,31.968599,-99.901813,65.214286
